# RAG with Langchain and Pinecone

This is a tutorial that I followed from James Briggs, here : https://youtu.be/LhnCsygAvzY?si=cJqyMVefEGa1s7uS

I didn't use his notebook directly, but typed it all in as he went. Found this to be worthwhile. I also reached out to my good friend, ChatGPT4, to ask about varoius things. Interestingly, ChatGPT doesn't really know about OpenAI embedding models or LangChain, but it is able to infer what's going on in the code and provide some really great explainers...

In [1]:
!pip install -qU langchain==0.0.292 openai==0.28.0 datasets==2.10.1 pinecone-client==2.2.4 tiktoken==0.5.1



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from langchain.chat_models import ChatOpenAI

apikey = os.getenv("OPENAI_API_KEY") or '<API-TOKEN-HERE>'

chat=ChatOpenAI(openai_api_key=apikey, model='gpt-3.5-turbo')

In [2]:
from langchain.schema import (SystemMessage, HumanMessage, AIMessage)
messages = [
    SystemMessage(content = 'You are a  helpful assistant.'),
    HumanMessage(content = 'Hi AI, how are you today?'),
    AIMessage(content = "I'm great thank you. How can I help you?"),
    HumanMessage(content = "I'd like to understand string theory.")
]

In [3]:
res = chat(messages)
res

AIMessage(content="String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It proposes that the fundamental building blocks of the universe are not particles, but rather tiny, vibrating strings. These strings can manifest as different particles depending on their vibrations.\n\nString theory also suggests the existence of extra dimensions beyond the familiar three spatial dimensions and one time dimension. The theory has the potential to explain phenomena such as gravity and the other fundamental forces of nature within a single framework.\n\nHowever, string theory is still a work in progress and has not yet been experimentally proven. It remains a topic of active research and debate among physicists. Let me know if you have any specific questions or if you'd like more information on any aspect of string theory.", additional_kwargs={}, example=False)

In [4]:
print(res.content)

String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It proposes that the fundamental building blocks of the universe are not particles, but rather tiny, vibrating strings. These strings can manifest as different particles depending on their vibrations.

String theory also suggests the existence of extra dimensions beyond the familiar three spatial dimensions and one time dimension. The theory has the potential to explain phenomena such as gravity and the other fundamental forces of nature within a single framework.

However, string theory is still a work in progress and has not yet been experimentally proven. It remains a topic of active research and debate among physicists. Let me know if you have any specific questions or if you'd like more information on any aspect of string theory.


In [5]:
messages.append(res)
prompt = HumanMessage(content="Why do physicists believe it can produce a 'unified theory'?")
messages.append(prompt)
res = chat(messages)
print(res.content)

Physicists are interested in string theory as a potential candidate for a unified theory because it has the capability to encompass all fundamental forces of nature within a single framework. In the standard model of particle physics, there are four fundamental forces: gravity, electromagnetism, the weak nuclear force, and the strong nuclear force. These forces are described by separate theories that do not currently fit together seamlessly.

String theory, on the other hand, has the potential to explain all these forces as manifestations of the same underlying principle— the vibrations of tiny strings in higher-dimensional space. By unifying these forces, physicists hope to achieve a more comprehensive understanding of the fundamental workings of the universe.

Additionally, string theory has the potential to address some of the unanswered questions in physics, such as the nature of gravity at quantum scales and the behavior of matter near black holes. It also provides a framework for

The following loads the dataset, llama-2-arxiv-papers-chunked. Link here : https://huggingface.co/datasets/jamescalam/llama-2-arxiv-papers-chunked

The first couple of rows of the dataset are as follows (with some columns removed from the table for brevity, along with fields truncated) :

|**doi**|**chunk-id**|**chunk**|id|title|summary|source|authors|categories|
|---|--------|-----|--|-----|-------|------|-------|----------|
|1102.0183|0|High-Performance Neural Networks for Visual Object Classi....|1102.0183|High-Performance Neural Networks....|We present a fast, fully parameterizable....|http://arxiv.org/pdf/1102.0183|[ "Dan C. Cireşan", "Ueli ....]|[ "cs.AI", "cs.NE" ]|12 pages, 2 figures, 5 tables|null|cs.AI|20110201|20110201|[]|
|1102.0183|1|January 2011 Abstract We present a fast, fully parameter....|1102.0183|High-Performance Neural Networks....|We present a fast, fully parameterizable....|http://arxiv.org/pdf/1102.0183|[ "Dan C. Cireşan", "Ueli....]|[ "cs.AI", "cs.NE" ]|12 pages, 2 figures, 5 tables|null|cs.AI|20110201|20110201|[]|

Note the first 3 columns - doi, chunk-id, chunk, source and title columns which are the columns we will be using to create the embedding vectors in pinecone database.

In [6]:
from datasets import load_dataset
dataset = load_dataset("jamescalam/llama-2-arxiv-papers-chunked", split="train")
dataset

c:\dev\learning2024\chatbotrag01\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 4838 examples [00:00, 52088.71 examples/s]


Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4838
})

In [7]:
import pinecone

pinecone.init(api_key='<PINECONE-API-KEY-HERE>', environment='gcp-starter')

## On dimension and cosine

The next bit of code sets up the vector database. Here's what ChatGPT says about dimension and why it matters :

The dimension `1536` in the context of setting up an index for embeddings, such as in Pinecone or similar vector databases, refers to the size of the vector that represents each text embedding. When you use an embedding model like "ada" from the LangChain or a similar framework, this model processes text and outputs a vector (a list of numbers) for each piece of text. The "dimension" is the length of this vector.

The specific number `1536` is tied to the architecture and design of the specific embedding model you're using. Each embedding model is trained to produce vectors of a fixed size, which captures the features or aspects of the text that the model considers important for understanding its meaning. In the case of the "ada" model you mentioned, it is designed to output vectors that are 1536 numbers long.

Here's why the dimension size matters:

1. **Consistency**: All vectors produced by the same model will have the same dimension. This consistency is crucial for storing, comparing, and retrieving embeddings within a vector database.

2. **Information Capacity**: The dimensionality of an embedding vector determines how much information about the text can be encoded. A higher dimension can capture more nuances and details of the text's meaning, but it also requires more computational resources to process and store.

3. **Model Design**: The choice of dimension size is part of the model's design and training process. It balances the need to capture detailed information about the text with the computational efficiency of processing the vectors.

4. **Compatibility**: When setting up a vector database or an index for storing embeddings, the dimension of the index must match the output dimension of the embedding model used. This ensures that all embeddings can be correctly stored and queried.

In summary, when you configure a Pinecone index with `dimension=1536`, you are specifying that this index is intended to store vectors of length 1536, matching the output of your embedding model. This allows the index to accurately store, search, and compare the text embeddings generated by your model, facilitating tasks like similarity search, clustering, or information retrieval based on the semantic meaning of the text.

Also... why cosine??

The `cosine` metric is chosen for measuring similarity between vectors in many machine learning and information retrieval tasks, including those involving text embeddings, because it effectively captures the orientation similarity between two vectors, independent of their magnitude. Here's a deeper look into why the `cosine` metric is often preferred, especially in the context of text embedding comparisons:

1. **Orientation Over Magnitude**: Cosine similarity focuses on the angle between two vectors rather than their length. This means that it measures how similar the directions of the vectors are, which is a good proxy for semantic similarity in text embeddings. Two documents with a similar thematic content will have embeddings that point in roughly the same direction, hence a smaller angle and a higher cosine similarity, regardless of the length of the vectors. This is particularly useful in text analysis because it allows for the comparison of texts of different lengths on a basis of their content's orientation in the vector space.

2. **Normalization**: Because cosine similarity is based on the orientation of the vectors, it inherently normalizes for length. This is important in applications where the frequency of words (which might affect the magnitude of a vector) is less important than the presence or absence of certain words or concepts. For instance, a longer document might naturally have larger magnitude embeddings due to more words, but that doesn't necessarily mean it's more relevant than a shorter document with a focused topic. Cosine similarity helps to adjust for these differences.

3. **Robustness in High-Dimensional Spaces**: Text embeddings often exist in high-dimensional spaces (like the 1536 dimensions you mentioned). In such spaces, the Euclidean distance (which considers both magnitude and direction) can become less intuitive due to the curse of dimensionality, where distances become inflated as dimensions increase. Cosine similarity, by focusing on the angle, remains a reliable measure of similarity in these high-dimensional settings.

4. **Efficiency and Effectiveness in Similarity Searches**: For tasks like finding the most relevant documents in a database, clustering documents based on content, or even recommendation systems, cosine similarity provides a balance between computational efficiency and effectiveness. It allows systems to quickly identify the most relevant vectors that are "closest" in direction to a query vector, facilitating efficient retrieval of similar items.

5. **Common Practice in NLP and Information Retrieval**: Cosine similarity is a well-established metric in the fields of NLP and information retrieval. Its widespread use comes from empirical evidence supporting its effectiveness in various tasks, including document similarity, sentiment analysis, and topic modeling. As such, it has become a standard choice, particularly when working with text embeddings.

In summary, the choice of the `cosine` metric is driven by its ability to capture semantic similarity through the orientation of vectors, its effectiveness in high-dimensional spaces, and its robustness across different lengths and scales of text data. This makes it particularly suited for applications involving text embeddings and semantic search.

In [8]:
import time

index_name = 'llama-2-rag'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)
index = pinecone.Index(index_name)

In [10]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In simple terms, when your code uses "text-embedding-ada-002" to create an embedding for RAG (Retrieval-Augmented Generation) for a language model, it's essentially using this pre-trained model to convert text into numerical vectors. These vectors are then used by the RAG model to understand and generate language based on the input text it receives. The embedding step is crucial because it allows the model to work with meaningful numerical representations of text, enabling it to perform tasks like answering questions, generating text, or even translating languages more effectively.

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=apikey)

In [14]:
texts = [
    "this is the first chunk of text",
    "then another second chunk of text is here"
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

In [19]:
# Import tqdm for showing progress bars during loops
from tqdm.auto import tqdm

# Convert the dataset into a pandas DataFrame for easier manipulation
data = dataset.to_pandas()

# Set the size of batches for processing to reduce memory usage
batch_size = 100

# Loop through the dataset in batches to process and index the data
for i in tqdm(range(0, len(data), batch_size)):
    # Calculate the end index for the current batch, ensuring it does not exceed the dataset's length
    i_end = min(len(data), i + batch_size)
    
    # Select the current batch from the dataset
    batch = data.iloc[i:i_end]
    
    # Generate unique identifiers for each document in the batch
    # using a combination of DOI and chunk ID
    ids = [f"{x['doi']} - {x['chunk-id']}" for i, x in batch.iterrows()]
    
    # Extract the text content of each document in the batch
    texts = [x['chunk'] for _, x in batch.iterrows()]
    
    # Use the embedding model to generate embeddings for each document's text content
    embeds = embed_model.embed_documents(texts)
    
    # Prepare metadata for each document in the batch, including the text content,
    # source, and title for additional context and searchability
    metadata = [
        {'text': x['chunk'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]

    # Insert or update the documents in the Pinecone vector database with the generated
    # identifiers, embeddings, and metadata for each document
    index.upsert(vectors=zip(ids, embeds, metadata))




100%|██████████| 49/49 [07:09<00:00,  8.76s/it]


In [20]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.04838,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

In [22]:
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(index, embed_model.embed_query, text_field)

c:\dev\learning2024\chatbotrag01\venv\lib\site-packages\langchain\vectorstores\pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [25]:
query = "What is so special about Llama 2?"

# The following line searches Pinecone vectorstore using a similarity search.
# The first parameter is the query above. Pinecone will first turn this query into a vector representation.
# The second parameter, 'k=3' specifies that we want the top 3 most similar results from the database.
# This means that similarity_search will return 3 items (or vectors) that are most similar to our query vector.
vectorstore.similarity_search(query, k=3)

[Document(page_content='Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic\nSergey Edunov Thomas Scialom\x03\nGenAI, Meta\nAbstract\nIn this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned\nlarge language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.\nOur ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our\nmodels outperform open-source chat models on most benchmarks we tested, and based on\nourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tuning and safety', metadata={'source': 'http://arxiv.org/pdf/2307.09288', 'title': 'Llama 2: Open Foundation and Fine-Tun

In [26]:
def augment_prompt(query: str):
    results = vectorstore.similarity_search(query, k=3)
    source_knowledge = "\n".join([x.page_content for x in results])
    
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""

    return augmented_prompt

In [27]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov Thomas Scialom
GenAI, Meta
Abstract
In this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
ourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tuning and safety
asChatGPT,BARD,andClaude. TheseclosedproductLLMsareheavilyﬁne-tunedtoalignwith

In [29]:
# create a new user prompt

prompt = HumanMessage(content = augment_prompt(query))
messages.append(prompt)
res = chat(messages)

print(res.content)

Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) that range in scale from 7 billion to 70 billion parameters. The fine-tuned LLMs, particularly the L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc models, are optimized for dialogue use cases. They have been shown to outperform open-source chat models on various benchmarks and have received positive evaluations for their helpfulness and safety.

What makes Llama 2 special is that these models generally perform better than existing open-source models and are on par with some closed-source models in terms of human evaluations. This suggests that Llama 2 could potentially serve as a suitable substitute for closed-source models in certain applications.


In [30]:
prompt = HumanMessage(content="what safety measures were used in the development of llama 2?")

res = chat(messages + [prompt])
print(res.content)

The safety measures used in the development of Llama 2 language models include detailed evaluations for helpfulness and safety. These evaluations were conducted to ensure that the models are suitable substitutes for closed-source models and outperform existing open-source chat models. Additionally, the Llama 2 models were optimized for dialogue use cases, aligning with human preferences to enhance usability and safety.

The developers also provided a detailed description of their approach to fine-tuning and safety, ensuring transparency and reproducibility in their methods. By focusing on safety and aligning with human preferences, the Llama 2 models aim to address the limitations of closed-source models and advance AI alignment research within the community.


In [31]:
prompt = HumanMessage(content=augment_prompt("What safety measures were used in the development of llama 2?"))

res = chat(messages + [prompt])
print(res.content)

In the development of Llama 2, safety measures were implemented to enhance the safety of the models. These safety measures included:

1. Using safety-specific data annotation and tuning: This involves annotating and tuning the data used to train the models to prioritize safety considerations and mitigate potential risks.

2. Conducting red-teaming: Red-teaming involves simulating adversarial attacks or scenarios to identify and address vulnerabilities in the models' safety protocols.

3. Employing iterative evaluations: Continuously evaluating the models through iterative processes to identify and rectify any safety issues that may arise during development.

Overall, these safety measures were crucial in ensuring that Llama 2 models were developed responsibly with a focus on safety, and they aimed to enable the community to reproduce fine-tuned LLMs and contribute to the ongoing improvement of model safety standards.
